# Exploratory Data Analysis (EDA): Bitext Retail Banking LLM Chatbot Training Dataset

This notebook performs **exploratory analysis and cleaning** for the Bitext retail banking dataset loaded directly from Hugging Face as a parquet file.

We will:
- Inspect the dataset (shape, columns, sample rows)
- Check **missing values** (including *blank strings*)
- Check **duplicates**
- Do **basic cleaning** (standardize columns, remove blanks, trim whitespace, drop duplicates)
- Explore **category/intent distribution** (class balance)
- Explore **text length statistics** (question/answer word counts)
- Extract **common n-grams** to understand vocabulary
- Save a cleaned copy for modeling

> Dataset source (parquet):  
> `hf://datasets/bitext/Bitext-retail-banking-llm-chatbot-training-dataset/bitext-retail-banking-llm-chatbot-training-dataset.parquet`

## 0) Setup

If you run this locally and you're missing packages, install them (uncomment the `pip install` line).

In [ ]:
# Uncomment if needed:
# %pip install pandas pyarrow fsspec huggingface-hub scikit-learn matplotlib

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## 1) Load the dataset

Using your exact loading snippet:

In [ ]:
import pandas as pd

df = pd.read_parquet(
    "hf://datasets/bitext/Bitext-retail-banking-llm-chatbot-training-dataset/"
    "bitext-retail-banking-llm-chatbot-training-dataset.parquet"
)

df.head()

## 2) Quick overview (shape, columns, types)

This helps confirm what's inside the parquet and the data types of each column.

In [ ]:
print("Rows, Columns:", df.shape)
display(pd.DataFrame({"column": df.columns, "dtype": [str(t) for t in df.dtypes]}))

df.describe(include="all").T.head(20)

## 3) Identify likely text / label columns

Different HF datasets use different names (e.g., `instruction/response`, `utterance/answer`, etc.).
This cell attempts to auto-detect:
- `question` (user input)
- `answer` (assistant output)
- `category` (department/topic label)
- `intent` (if present)

In [ ]:
TEXT_CANDIDATES = ["instruction", "question", "utterance", "query", "input", "prompt", "text", "user"]
ANSWER_CANDIDATES = ["response", "answer", "assistant", "output", "completion", "bot"]
CATEGORY_CANDIDATES = ["category", "dept", "department", "topic", "label"]
INTENT_CANDIDATES = ["intent", "scenario", "intent_name"]

def pick_col(columns, candidates):
    cols_lower = {c.lower(): c for c in columns}
    for cand in candidates:
        if cand.lower() in cols_lower:
            return cols_lower[cand.lower()]
    # fallback heuristic
    for c in columns:
        cl = c.lower()
        if any(k in cl for k in ["question", "instruction", "utterance", "query", "prompt", "text", "input"]):
            return c
    return None

qcol = pick_col(df.columns, TEXT_CANDIDATES)
acol = pick_col(df.columns, ANSWER_CANDIDATES)
catcol = pick_col(df.columns, CATEGORY_CANDIDATES)
intentcol = pick_col(df.columns, INTENT_CANDIDATES)

print("Detected columns:")
print("  question:", qcol)
print("  answer:  ", acol)
print("  category:", catcol)
print("  intent:  ", intentcol)

## 4) Missing values (including blank strings)

For text datasets, **blanks** often matter as much as `NaN`.
This section checks:
- True missing (`NaN`)
- Blank strings like `""` or `"   "`

In [ ]:
def is_blank(x):
    if x is None:
        return True
    # pandas NA
    try:
        if pd.isna(x):
            return True
    except Exception:
        pass
    return str(x).strip() == ""

missing_summary = []
for c in df.columns:
    na_rate = df[c].isna().mean()
    blank_rate = df[c].apply(is_blank).mean() if df[c].dtype == "object" else np.nan
    missing_summary.append({
        "column": c,
        "na_%": round(na_rate * 100, 2),
        "blank_%": round(blank_rate * 100, 2) if not np.isnan(blank_rate) else None
    })

missing_df = pd.DataFrame(missing_summary).sort_values(["na_%","blank_%"], ascending=False)
display(missing_df)

# Quick bar chart of blank% for object columns (if any)
obj_missing = missing_df.dropna(subset=["blank_%"]).copy()
if not obj_missing.empty:
    fig = plt.figure()
    plt.bar(obj_missing["column"], obj_missing["blank_%"])
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Blank %")
    plt.title("Blank strings per column (object columns)")
    plt.tight_layout()
    plt.show()

## 5) Duplicates

We check:
- Duplicate rows (exact duplicates across all columns)
- Duplicate questions (common in prompt-response datasets)

In [ ]:
dup_rows = df.duplicated().mean()
print(f"Exact duplicate rows: {dup_rows:.2%}")

if qcol:
    dup_q = df[qcol].astype(str).duplicated().mean()
    print(f"Duplicate questions in '{qcol}': {dup_q:.2%}")

## 6) Build a clean modeling dataframe

We standardize to a consistent schema:

- `question` (string)
- `answer` (string)
- `category` (UPPERCASE label if available; otherwise 'CONTACT')
- `intent` (optional; blank if not available)

Cleaning steps:
- Convert to strings
- Strip whitespace & collapse multiple spaces
- Drop rows where question/answer are blank
- Drop duplicates (question+answer)

In [ ]:
if not qcol or not acol:
    raise ValueError(f"Could not detect question/answer columns. Found columns: {list(df.columns)}")

base = df.copy()

base = base.rename(columns={
    qcol: "question",
    acol: "answer",
    (catcol if catcol else "___no_category___"): "category",
    (intentcol if intentcol else "___no_intent___"): "intent",
})

# Ensure required columns exist
if "category" not in base.columns:
    base["category"] = "CONTACT"
if "intent" not in base.columns:
    base["intent"] = ""

# Normalize text
def norm_text(s):
    s = "" if s is None else str(s)
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

base["question"] = base["question"].apply(norm_text)
base["answer"] = base["answer"].apply(norm_text)

# Category / intent cleanup
base["category"] = base["category"].astype(str).apply(norm_text)
base["category"] = base["category"].replace({"": "CONTACT"}).str.upper()

base["intent"] = base["intent"].astype(str).apply(norm_text)
# keep intent as-is (case can matter), but replace pure blanks to ""
base.loc[base["intent"].str.strip() == "", "intent"] = ""

# Drop blank Q/A
before = len(base)
base = base[(base["question"].str.strip() != "") & (base["answer"].str.strip() != "")]
after_drop_blank = len(base)

# Drop duplicates
base = base.drop_duplicates(subset=["question", "answer"]).reset_index(drop=True)
after_dedup = len(base)

print("Rows before cleaning:", before)
print("After dropping blank Q/A:", after_drop_blank)
print("After dropping duplicate Q/A:", after_dedup)

base.head()

## 7) Re-check missing values after cleaning

We re-run missing checks for the new `base` dataset.

In [ ]:
missing_after = []
for c in base.columns:
    na_rate = base[c].isna().mean()
    blank_rate = base[c].apply(is_blank).mean() if base[c].dtype == "object" else np.nan
    missing_after.append({
        "column": c,
        "na_%": round(na_rate * 100, 2),
        "blank_%": round(blank_rate * 100, 2) if not np.isnan(blank_rate) else None
    })

missing_after_df = pd.DataFrame(missing_after).sort_values(["na_%","blank_%"], ascending=False)
display(missing_after_df)

## 8) Category distribution (class balance)

This is critical for a chatbot routing/classification task:
- If some categories are rare, you may need **class weighting**, **sampling**, or **merging**.

In [ ]:
cat_counts = base["category"].value_counts().rename_axis("category").reset_index(name="count")
display(cat_counts)

fig = plt.figure()
plt.bar(cat_counts["category"], cat_counts["count"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Count")
plt.title("Examples per category")
plt.tight_layout()
plt.show()

# Show rare categories (adjust threshold if needed)
rare_threshold = 50
rare = cat_counts[cat_counts["count"] < rare_threshold]
print(f"Rare categories (<{rare_threshold} examples):", len(rare))
display(rare)

## 9) Intent distribution (if available)

Some versions include an `intent`/scenario column. We inspect it if present.

In [ ]:
if "intent" in base.columns and (base["intent"].str.strip() != "").any():
    intent_counts = base[base["intent"].str.strip() != ""]["intent"].value_counts().head(30)
    display(intent_counts.rename_axis("intent").reset_index(name="count"))

    fig = plt.figure()
    intent_counts.head(20).plot(kind="bar")
    plt.ylabel("Count")
    plt.title("Top 20 intents")
    plt.tight_layout()
    plt.show()
else:
    print("No non-empty intent values found (or intent column not present).")

## 10) Text length analysis (words & characters)

This helps you decide:
- truncation strategy (if you train models)
- detect outliers (very long/very short prompts)

In [ ]:
def word_count(s):
    return len(re.findall(r"[A-Za-z0-9']+", str(s).lower()))

base["_q_words"] = base["question"].apply(word_count)
base["_a_words"] = base["answer"].apply(word_count)
base["_q_chars"] = base["question"].astype(str).apply(len)
base["_a_chars"] = base["answer"].astype(str).apply(len)

display(base[["_q_words","_a_words","_q_chars","_a_chars"]].describe(percentiles=[.5,.9,.95,.99]).T)

# histograms (clipped at p99 to avoid a single huge outlier ruining the plot)
q_clip = base["_q_words"].clip(upper=base["_q_words"].quantile(0.99))
a_clip = base["_a_words"].clip(upper=base["_a_words"].quantile(0.99))

fig = plt.figure()
plt.hist(q_clip, bins=50)
plt.title("Question length (words) — clipped at p99")
plt.xlabel("Words"); plt.ylabel("Count")
plt.tight_layout()
plt.show()

fig = plt.figure()
plt.hist(a_clip, bins=50)
plt.title("Answer length (words) — clipped at p99")
plt.xlabel("Words"); plt.ylabel("Count")
plt.tight_layout()
plt.show()

## 11) Inspect outliers and edge cases

We print:
- shortest questions
- longest questions
- longest answers

This helps you spot data artifacts and decide if additional cleaning is needed.

In [ ]:
def show_examples(df, col, n=8, ascending=True, title=""):
    sub = df.sort_values(col, ascending=ascending).head(n)[["category","question","answer",col]]
    st = f"\n{title}\n" if title else ""
    print(st)
    display(sub)

show_examples(base, "_q_words", n=10, ascending=True, title="Shortest questions")
show_examples(base, "_q_words", n=10, ascending=False, title="Longest questions")
show_examples(base, "_a_words", n=10, ascending=False, title="Longest answers")

## 12) Common n-grams in questions (overall)

This is a simple way to understand what customers ask about most often.
We compute top unigrams + bigrams using `CountVectorizer`.

> Tip: If you see sensitive terms (PIN/password), you can later add safety filters.

In [ ]:
texts = base["question"].astype(str).tolist()

vec = CountVectorizer(stop_words="english", ngram_range=(1,2), min_df=10)
X = vec.fit_transform(texts)
counts = np.asarray(X.sum(axis=0)).ravel()
vocab = np.array(vec.get_feature_names_out())
topn = 30
idx = counts.argsort()[-topn:][::-1]

ngrams = pd.DataFrame({"ngram": vocab[idx], "count": counts[idx]})
display(ngrams)

## 13) N-grams by category (top categories)

For routing/classification, it’s useful to see the phrases that characterize each category.
We show top n-grams for the **top 5 largest categories**.

In [ ]:
top_cats = base["category"].value_counts().head(5).index.tolist()
results = {}

for cat in top_cats:
    subset = base[base["category"] == cat]["question"].astype(str).tolist()
    if len(subset) < 50:
        continue
    v = CountVectorizer(stop_words="english", ngram_range=(1,2), min_df=5)
    Xc = v.fit_transform(subset)
    counts = np.asarray(Xc.sum(axis=0)).ravel()
    vocab = np.array(v.get_feature_names_out())
    idx = counts.argsort()[-20:][::-1]
    results[cat] = pd.DataFrame({"ngram": vocab[idx], "count": counts[idx]})

for cat, table in results.items():
    print(f"\nTop n-grams for category: {cat}")
    display(table.head(20))

## 14) (Optional) TF-IDF keywords per category

TF‑IDF highlights terms that are more unique to a category compared to the whole dataset.
This can help you:
- justify why categories are separable
- create keyword rules for a baseline router

In [ ]:
tfidf = TfidfVectorizer(stop_words="english", ngram_range=(1,2), min_df=5, max_df=0.9)
X = tfidf.fit_transform(base["question"].astype(str))
features = np.array(tfidf.get_feature_names_out())

def top_tfidf_for_category(cat, k=15):
    idx = base.index[base["category"] == cat].to_numpy()
    if len(idx) == 0:
        return pd.DataFrame(columns=["term","score"])
    v = np.asarray(X[idx].mean(axis=0)).ravel()
    top = v.argsort()[-k:][::-1]
    return pd.DataFrame({"term": features[top], "score": v[top]})

for cat in top_cats:
    print(f"\nTF‑IDF keywords for: {cat}")
    display(top_tfidf_for_category(cat, k=15))

## 15) Save the cleaned dataset

You can now save `base` for downstream tasks:
- training a TF‑IDF retrieval bot
- training a classifier for routing
- creating train/validation splits

In [ ]:
# Save locally
base.to_parquet("bitext_retail_banking_clean.parquet", index=False)
base.to_csv("bitext_retail_banking_clean.csv", index=False)

print("Saved:")
print(" - bitext_retail_banking_clean.parquet")
print(" - bitext_retail_banking_clean.csv")

## 16) Key takeaways (write these into your assignment)

When you finish running this notebook, summarize:
- Whether there were missing values / blank strings
- Whether duplicates existed and what you removed
- Category balance (which categories dominate, which are rare)
- Typical question/answer lengths (median and long-tail)
- Example vocabulary signals (top n-grams / TF-IDF terms)
- What cleaning decisions you made and why

**Next step suggestions**:
- If class imbalance is strong: use class weights, stratified splits, or merge rare categories.
- If many very short questions exist: include bigrams, maybe character n-grams.
- If outliers are extreme: cap length or remove noisy rows.